Introduction
==============
This case study aims to give you an idea of applying EDA in a real business scenario. In this case study, apart from applying the techniques that you have learnt in the EDA module, you will also develop a basic understanding of risk analytics in banking and financial services and understand how data is used to minimise the risk of losing money while lending to customers.

 

Business Understanding
======================
The loan providing companies find it hard to give loans to the people due to their insufficient or non-existent credit history. Because of that, some consumers use it as their advantage by becoming a defaulter. Suppose you work for a consumer finance company which specialises in lending various types of loans to urban customers. You have to use EDA to analyse the patterns present in the data. This will ensure that the applicants capable of repaying the loan are not rejected.

 

When the company receives a loan application, the company has to decide for loan approval based on the applicant’s profile. Two types of risks are associated with the bank’s decision:

If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company

If the applicant is not likely to repay the loan, i.e. he/she is likely to default, then approving the loan may lead to a financial loss for the company.

 

The data given below contains the information about the loan application at the time of applying for the loan. It contains two types of scenarios:

The client with payment difficulties: he/she had late payment more than X days on at least one of the first Y instalments of the loan in our sample,

All other cases: All other cases when the payment is paid on time.

 

 

When a client applies for a loan, there are four types of decisions that could be taken by the client/company):

##### Approved: The Company has approved loan Application

##### Cancelled: The client cancelled the application sometime during approval. Either the client changed her/his mind about the loan or in some cases due to a higher risk of the client he received worse pricing which he did not want.

##### Refused: The company had rejected the loan (because the client does not meet their requirements etc.).

##### Unused offer:  Loan has been cancelled by the client but on different stages of the process.

In this case study, you will use EDA to understand how consumer attributes and loan attributes influence the tendency of default.

 

 

Business Objectives
===================
This case study aims to identify patterns which indicate if a client has difficulty paying their installments which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc. This will ensure that the consumers capable of repaying the loan are not rejected. Identification of such applicants using EDA is the aim of this case study.

 

In other words, the company wants to understand the driving factors (or driver variables) behind loan default, i.e. the variables which are strong indicators of default.  The company can utilise this knowledge for its portfolio and risk assessment.

To develop your understanding of the domain, you are advised to independently research a little about risk analytics - understanding the types of variables and their significance should be enough).

# PART I : DATA SOURCING
----------------------------------------------------------

# 1. Importing the libraries and files



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 

from mpl_toolkits.mplot3d import Axes3D 
from plotly.offline import init_notebook_mode, iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import plotly.express as px
import plotly.graph_objs as go
import plotly
from plotly import tools
import plotly.express as px
from scipy.stats import boxcox
init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 400) # Used for displaying all the columns
pd.set_option('display.max_rows', 400) # Used for displaying all the rows
pd.set_option('display.width', 1200) # Used for displaying the cells

import warnings
warnings.filterwarnings('ignore')


1.2 Importing the Data Files
================================================

In [ ]:
#importing the data from local
application_data = pd.read_csv('application_data.csv')
previous_application = pd.read_csv('previous_application.csv')

Loading & displaying the data
=============

## 2. Display of Data


In [ ]:
#Reading the application_data csv file
application_data.head()

In [ ]:
#Reading the Previous Application _data csv file
previous_application.head()

# PART B : DATA CLEANING
----------------------------------------------------------
> Fixing Rows and Columns
>
> Imputation of Values for Null and Missing Columns
>
> Handling Outlier
>
> Standerization of Data
>
> Fixing Invalid Values and Filter Data

2.1 Inspecting the DataFrame
==============



2.2 Inspecting Application Data
-------------------------------------------------

In [ ]:
# Determining the shape of the datset: Checking the number of rows and columns in the dataframe
application_data.shape

In [ ]:
application_data.info()

In [ ]:
# Check the summary for the numeric columns
application_data.describe()

In [ ]:
# Percentage of null values in each column
null_percent = application_data.isnull().sum()/len(application_data)*100

In [ ]:
null_percent.sort_values(ascending = False).head(50)

2.3 Inspecting Previous Application Data
-------------------------------------------------

In [ ]:
# Determining the shape of the datset: Checking the number of rows and columns in the dataframe
previous_application.shape

In [ ]:
previous_application.info()

In [ ]:
# Check the summary for the numeric columns
previous_application.describe()

### 2.4 Null Values Checking and Fixing

In [ ]:
# List of Columns & NA counts where NA or null values are more than 35%
NullCol = application_data.isnull().sum().sort_values(ascending = False)
NullCol = NullCol[NullCol.values >(0.35*len(application_data))]
plt.figure(figsize=(20,4))
NullCol.plot(kind='bar', color="#4CB391")
plt.title('List of Columns & NA counts where NA/Null values are more than 35%')
plt.show()

In [ ]:
# Number of columns having percentage of null values > 35%
len(NullCol)

In [ ]:
# Function to remove the null columns having percentage > 35%
def removeNulls(dataframe,percent=0.35):
    df = dataframe.copy()
    ishape = df.shape
    colnames = (df.isnull().sum()/len(df))
    colnames = list(colnames[colnames.values>=percent].index)
    df.drop(labels = colnames,axis =1,inplace=True)        
    print("Number of Columns dropped\t: ",len(colnames))    
    print("\nOld dataset rows,columns",ishape,"\nNew dataset rows,columns",df.shape)
    return df


In [ ]:
# Comparision of shape of new and old data sets
application_data = removeNulls(application_data,percent = 0.35)

In [ ]:
# Checking the % of null values for each column in new dataset
null = application_data.isnull().sum()/len(application_data)*100
null.sort_values(ascending = False).head(60)

### 3  Imputation of Missing Values In Missing data of some Columns

### Checking for values to impute in columns

In [ ]:
application_data['EXT_SOURCE_2'].value_counts()

In [ ]:
# EXT_SOURCE_2 is a continuous variable. So checking for outliers
plt.style.use('ggplot')
plt.figure(figsize=[10,5])
sns.boxplot(application_data['EXT_SOURCE_2'])
plt.show()

In [ ]:
# Since EXT_SOURCE_2 has no outlier, we can choose mean to impute the column
imputVAL = round(application_data['EXT_SOURCE_2'].mean(),2)
print(f'Since EXT_SOURCE_2 has no outlier, the column can be imputed using the mean of the coumn i.e. {imputVAL}')

#### 3.1 Imputating the missing values for the column 'OCCUPATION_TYPE'

In [ ]:
application_data.OCCUPATION_TYPE.value_counts()

In [ ]:
# Since AMT_ANNUITY is a continuous variable. So checking for outliers
sns.boxplot(application_data['AMT_ANNUITY'])
plt.show()

In [ ]:
imputVAL = application_data.OCCUPATION_TYPE.mode()
print(f'Clearly the column NAME_TYPE_SUITE is a categorical column. So this column can be imputed using the mode of the column i.e {imputVAL[0]}') 

#### 3.2 Imputating the missing values for the column NAME_TYPE_SUITE 

In [ ]:
application_data['NAME_TYPE_SUITE'].value_counts()

In [ ]:
imputVAL = application_data['NAME_TYPE_SUITE'].mode()
print(f'Clearly the column NAME_TYPE_SUITE is a categorical column. So this column can be imputed using the mode of the column i.e {imputVAL[0]}') 

#### 3.3 Imputating the missing values for the column 'AMT_REQ_CREDIT_BUREAU_YEAR' with mode, where value= '0'


In [ ]:
application_data.AMT_REQ_CREDIT_BUREAU_YEAR.value_counts()

In [ ]:
# Since AMT_REQ_CREDIT_BUREAU_YEAR is a continuous variable. So checking for outliers
sns.boxplot(application_data['AMT_REQ_CREDIT_BUREAU_YEAR'])
plt.show()

In [ ]:
imputVAL = application_data['AMT_REQ_CREDIT_BUREAU_YEAR'].mode()
print(f'Clearly the column NAME_TYPE_SUITE is a categorical column. So this column can be imputed using the mode of the column i.e {imputVAL[0]}') 

#### 3.4 Imputating the missing values for the column few more columns with the mode, which is the value '0'. Replacing missing values with the most frequent value would make more sense.

In [ ]:
application_data.AMT_REQ_CREDIT_BUREAU_MON.mode()

In [ ]:
application_data.AMT_REQ_CREDIT_BUREAU_WEEK.mode()

In [ ]:
application_data.AMT_REQ_CREDIT_BUREAU_DAY.mode()

In [ ]:
application_data.AMT_REQ_CREDIT_BUREAU_HOUR.mode()


In [ ]:
application_data.AMT_REQ_CREDIT_BUREAU_QRT.mode()

#### 3.5 For the column 'AMT_GOODS_PRICE', imputing the missing values with the median,where value = '450000'

In [ ]:
application_data.AMT_GOODS_PRICE.value_counts() 

In [ ]:
#Finding the median
application_data.AMT_GOODS_PRICE.median()

#### 3.6 For the column 'AMT_ANNUITY', we can impute the missing values with the median, where value= '24903.0'

In [ ]:
application_data.AMT_ANNUITY.value_counts()

In [ ]:
#Finding the median
application_data.AMT_ANNUITY.median()

#### 3.7 Errors in Data types and Data

#### Imputing the values of columns having negative or mixed values with absolute values

In [ ]:
# Checking the values present in columns starting with 'DAYS'
print(application_data['DAYS_BIRTH'].unique())
print(application_data['DAYS_EMPLOYED'].unique())
print(application_data['DAYS_REGISTRATION'].unique())
print(application_data['DAYS_ID_PUBLISH'].unique())
print(application_data['DAYS_LAST_PHONE_CHANGE'].unique())

In [ ]:
# The columns starting with 'DAYS'
filter_col = [col for col in application_data if col.startswith('DAYS')]
filter_col

In [ ]:
# Applying abs() function to columns starting with 'DAYS' to convert the negative values to positive
application_data[filter_col]= abs(application_data[filter_col])

In [ ]:
print(application_data['DAYS_BIRTH'].unique())
print(application_data['DAYS_EMPLOYED'].unique())
print(application_data['DAYS_REGISTRATION'].unique())
print(application_data['DAYS_ID_PUBLISH'].unique())
print(application_data['DAYS_LAST_PHONE_CHANGE'].unique())

In [ ]:
application_data.describe()

In [ ]:
#Checking the Gender Columns Number
application_data.CODE_GENDER.value_counts()

In [ ]:
#Imputing the value'XNA' which means not available for the column 'CODE_GENDER' with value "F"
application_data.loc[application_data.CODE_GENDER == 'XNA','CODE_GENDER'] = 'F'
application_data.CODE_GENDER.value_counts()

In [ ]:
#Checking the Organization Types Present in the dataset
application_data.ORGANIZATION_TYPE.value_counts()

In [ ]:
# Imputing the value'XNA' which means not available for the column 'ORGANIZATION_TYPE' with 'NaN'
application_data = application_data.replace('XNA',np.NaN)

In [ ]:
application_data.ORGANIZATION_TYPE.value_counts()

#### 3.8 Binning of continuous variables

In [ ]:
# Binning 'AMT_INCOME_RANGE' based on quantiles
application_data['AMT_INCOME_RANGE'] = pd.qcut(application_data.AMT_INCOME_TOTAL, q=[0, 0.2, 0.5, 0.8, 0.95, 1], labels=['VERY_LOW', 'LOW', "MEDIUM", 'HIGH', 'VERY_HIGH'])
application_data['AMT_INCOME_RANGE'].head(11)

In [ ]:
# Binning 'AMT_CREDIT_RANGE' by creating various ranges
application_data['AMT_CREDIT_RANGE'] = pd.qcut(application_data.AMT_CREDIT, q=[0, 0.2, 0.5, 0.8, 0.95, 1], labels=['VERY_LOW', 'LOW', "MEDIUM", 'HIGH', 'VERY_HIGH'])
application_data['AMT_CREDIT_RANGE'].head(11)

In [ ]:
application_data['AMT_CREDIT_RANGE'].head(10)

In [ ]:
# Converting 'DAYS_BIRTH' to years
application_data['DAYS_BIRTH']= (application_data['DAYS_BIRTH']/365).astype(int)

In [ ]:
application_data['DAYS_BIRTH'].unique()

In [ ]:
# Biining 'DAYS_BIRTH'
application_data['DAYS_BIRTH_BINS']=pd.cut(application_data['DAYS_BIRTH'], bins=[19,25,35,60,100], labels=['Very_Young','Young', 'Middle_Age', 'Senior_Citizen'])

In [ ]:
application_data['DAYS_BIRTH_BINS'].value_counts()

## 4. Data Cleaning

### 4.1 Finding Outliers

In [ ]:
# boxplot for 'AMT_INCOME_TOTAL' column to check the outliers
fig = px.box(application_data, y="AMT_INCOME_TOTAL",title='Distribution of AMT_INCOME_TOTAL' )
fig.show()

Here, in the coloumn 'AMT_INCOME_TOTAL' which tells us the income of the client. We observe a value of 117M which is surely an outlier.

In [ ]:
# distribution of 'DAYS_EMPLOYED'
sns.set(rc={'figure.figsize':(20,10)})
sns.distplot(application_data['DAYS_EMPLOYED'], color="#4CB391" )
plt.title('Distribution of DAYS_EMPLOYED', fontdict={'fontsize':20} )
plt.show()

Here, in the coloumn 'DAYS_EMPLOYED' which tells how many days before the application the person started current employment. We observe a value which is greater that 20,000 which is surely an outlier because 25,000/365 will be around 54 years. Considering that a person started working at age of 21, the person will be 21+54 will turn out to be 75 years old.

In [ ]:
fig = px.box(application_data, y="DAYS_BIRTH",title='Distribution of DAYS_BIRTH' )
fig.show()

We do not find any outliers in the coloumn 'DAYS_BIRTH'.

In [ ]:
fig = px.box(application_data, y="AMT_ANNUITY",title='Distribution of AMT_ANNUITY' )
fig.show()

Here, in the coloumn 'AMT_AANNUITY' which tells the loan annuity. We observe a value which is greater that 258000 which is surely an outlier.

In [ ]:
fig = px.box(application_data, y="DAYS_REGISTRATION", title='Distribution of DAYS_REGISTRATION' )
fig.show()

Here, in the coloumn 'DAYS_REGISTRATION' which how many days before the application did client change his registration. We observe a value which is greater that 24000 which is surely an outlier.

### 4.2 Checking the distribution

In [ ]:
# Distribution of 'OCCUPATION_TYPE'
temp = application_data["OCCUPATION_TYPE"].value_counts()
temp.iplot(kind='bar', xTitle = 'Occupation', yTitle = "Count", title = 'Occupation of Applicant\'s who applied for loan', color = 'green')

We can observer that Labourers, Sales Staff and Core Staff constitute the majority whereas IT Staff is on the lower side.

In [ ]:
# Distribution of 'ORGANIZATION_TYPE'
temp = application_data["ORGANIZATION_TYPE"].value_counts()
temp.iplot(kind='bar', xTitle = 'Organization Name', yTitle = "Count", title = 'Types of Organizations who applied for loan ', color = 'red')

We can observe that Business Entity Type 3 organizations have majority of the loan applications.

### 4.3 Checking Imbalance

In [ ]:
im = application_data["TARGET"].value_counts()
df =pd.DataFrame({'labels': im.index,'values': im.values})
df.iplot(kind='pie',labels='labels',values='values', title='Target Imbalance Distribution', hole = 0.5)

The imbalance is high between target variables.
> Its clear that there is an imbalance between people who defaulted and who didn't default. More than 92% of people didn't default as opposed to 8% who defaulted.

### 4.4 Dividing the dataset into two different datasets based upon 'Target' value.

In [ ]:
# Dividing the original dataset into two different datasets depending upon the target value
target0 = application_data.loc[application_data.TARGET == 0]
target1 = application_data.loc[application_data.TARGET == 1]

# PART C : UNIVARIATE ANALYSIS
----------------------------------------------------------
> univariate analysis is nothing but an analysis of one particular variable at a time. 
> It is important to look at each and every variable and perform analysis on it

## 5. Univariate Analysis of Categorical Variables

#### Gender Distribution

In [ ]:
# Gender Distibution of Loan Non-Payment Difficulties
a0 = target0["CODE_GENDER"].value_counts()
df = pd.DataFrame({'labels': a0.index,'values': a0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Gender Distibution of Loan- Non Payment Difficulties', hole = 0.5)

In [ ]:
# Gender Distibution of Loan Payment Difficulties
a1 = target1["CODE_GENDER"].value_counts()
df = pd.DataFrame({'labels': a1.index,'values': a1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Gender Distribution of Loan Payment Difficulties', hole = 0.5)

>Comparing the Payment Difficulties and Non Payment Difficulties on the basis of Gender, we observe that Females are the majority in both the cases although there is an increase in the percentage in Male Payment Difficulties from Non-Payment Difficulties
> We can see that Female contribute 67% to the non-defaulters while 57% to the defaulters. We can conclude that
> <br> We see more female applying for loans than males and hence the more number of female defaulters as well. 
> <br> **But the rate of defaulting of FEMALE is much lower compared to their MALE counterparts.**

#### Income Source

In [ ]:
# Income sources of Loan- Non Payment Difficulties
b0 = target0["NAME_INCOME_TYPE"].value_counts()
df = pd.DataFrame({'labels': b0.index,'values': b0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Income sources of Loan- Non Payment Difficulties', hole = 0.5)

In [ ]:
# Income sources of Loan Payment Difficulties
b1 = target1["NAME_INCOME_TYPE"].value_counts()
df = pd.DataFrame({'labels': b1.index,'values': b1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Income sources of Loan Payment Difficulties', hole = 0.5)

>We observe a decrease in the percentage of Payment Difficulties who are pentioners and an increase in the percentage of Payment Difficulties who are working when compared the percentages of both Payment Difficulties and non-Payment Difficulties.

> We can notice that the students don't default. The reason could be they are not required to pay during the time they are students.
<br> We can also see that the BusinessMen never default.
<br>Most of the loans are distributed to working class people<br>We also see that working class people contribute 51% to non defaulters while they contribute to 61% of the defaulters. Clearly, the chances of defaulting are more in their case.

#### Family Status

In [ ]:
# Family Status of Loan- Non Payment Difficulties
c0 = target0["NAME_FAMILY_STATUS"].value_counts()
df = pd.DataFrame({'labels': c0.index,'values': c0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Family Status of Loan- Non Payment Difficulties', hole = 0.5)

In [ ]:
# Family Status of Loan Payment Difficulties
c1 = target1["NAME_FAMILY_STATUS"].value_counts()
df = pd.DataFrame({'labels': c1.index,'values': c1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Family Status of Loan Payment Difficulties', hole = 0.5)

> We observe a decrease in the percentage of married and widowed with Loan Payment Difficulties and an increase in the the percentage of single and civil married with Loan Payment Difficulties when comapred with the percentages of both Loan Payment Difficulties and Loan Non-Payment Difficulties


#### Education

In [ ]:
# Education of Loan- Non Payment Difficulties
d0 = target0["NAME_EDUCATION_TYPE"].value_counts()
df = pd.DataFrame({'labels': d0.index,'values': d0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Education of Loan- Non Payment Difficulties', hole = 0.5)

In [ ]:
# Education of Loan Payment Difficulties
d1 = target1["NAME_EDUCATION_TYPE"].value_counts()
df = pd.DataFrame({'labels': d1.index,'values': d1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Education of Loan Payment Difficulties', hole = 0.5)

>>We observe an increase in percentage of Loan Payment Difficulties whose educational qualifications are secondary/secondary special and a decrease in the percentage of Loan Payment Difficulties who have completed higher education when compared with the percentages of Loan Payment Difficulties and Loan Non-Payment Difficulties

>
>> Almost all of the Education categories are equally likely to default except for the higher educated ones who are less likely to default and secondary educated people are more likely to default

#### Housing

In [ ]:
# Type of House of Loan- Non Payment Difficulties
e0 = target0["NAME_HOUSING_TYPE"].value_counts()
df = pd.DataFrame({'labels': e0.index,'values': e0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Type of House of Loan-Non Payment Difficulties', hole = 0.5)

In [ ]:
# Type of House of Payment Difficulties
e1 = target1["NAME_HOUSING_TYPE"].value_counts()
df = pd.DataFrame({'labels': e1.index,'values': e1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Type of House of Loan Payment Difficulties', hole = 0.5)

We observe an increase in the percentage of Payment Difficulties who live with their parents when compared to the percentages of Payment Difficulties and non-Payment Difficulties
> It is clear from the chart that people who have House/Appartment, tend to apply for more loans.
> <br>People living with parents tend to default more often when compared with others.The reason could be their living expenses are more due to their parents living with them. 

#### Income Range

In [ ]:
# Income range of Loan-Non Payment Difficulties
f0 = target0["AMT_INCOME_RANGE"].value_counts()
df = pd.DataFrame({'labels': f0.index,'values': f0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Income range of Loan-Non Payment Difficulties', hole = 0.5)

In [ ]:
# Income range of Loan Payment Difficulties
f1 = target1["AMT_INCOME_RANGE"].value_counts()
df = pd.DataFrame({'labels': f1.index,'values': f1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Income range of Loan Payment Difficulties', hole = 0.5)

We observe an increase in the percentage of Loan Payment Difficulties whose income is low when compared with the percentages of Payment Difficulties and Loan-Non Payment Difficulties


#### Age

In [ ]:
# Age of Loan-Non Payment Difficulties
g0 = target0["DAYS_BIRTH_BINS"].value_counts()
df = pd.DataFrame({'labels': g0.index,'values': g0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Age of Loan-Non Payment Difficulties', hole = 0.5)

In [ ]:
# Age of Loan Payment Difficulties
g1 = target1["DAYS_BIRTH_BINS"].value_counts()
df = pd.DataFrame({'labels': g1.index,'values': g1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Age of Loan Payment Difficulties', hole = 0.5)

>We observe that there is an increase in the percentage of Loan Payment Difficulties who are young in age when compared to the percentages of Payment Difficulties and Loan-Non Payment Difficulties.
> We see that (25,30] age group tend to default more often. So they are the riskiest people to loan to.
> <br> With increasing age group, people tend to default less starting from the age 25. One of the reasons could be they get employed around that age and with increasing age, their salary also increases.


#### Type of Loan

In [ ]:
# Types of Loans taken by Loan-Non Payment Difficulties 
h0 = target0["NAME_CONTRACT_TYPE"].value_counts()
df = pd.DataFrame({'labels': h0.index,'values': h0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Types of Loans taken by Loan-Non Payment Difficulties', hole = 0.5)

In [ ]:
# Types of Loans taken by Loan Payment Difficulties
h1 = target1["NAME_CONTRACT_TYPE"].value_counts()
df = pd.DataFrame({'labels': h1.index,'values': h1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Types of Loans taken by Loan Payment Difficulties', hole = 0.5)

We can observe that cash loans are preffered by both Loan Payment Difficulties and Loan-Non Payment Difficulties although there is a decrease in the percentage of Payment Difficulties who opt for revolving loans. 

#### Accompanyed By

In [ ]:
# Loan-Non Payment Difficulties are Accompanied by
i0 = target0["NAME_TYPE_SUITE"].value_counts()
df = pd.DataFrame({'labels': i0.index,'values': i0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Loan-Non Payment Difficulties are Accompanyed by', hole = 0.5)

In [ ]:
# Loan Payment Difficulties are Accompanied by
i1 = target1["NAME_TYPE_SUITE"].value_counts()
df = pd.DataFrame({'labels': i1.index,'values': i1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Loan Payment Difficulties are Accompanyed by', hole = 0.5)

We cannot observe any major changes here.

#### Organization Type

In [ ]:
# Types of Organizations who applied for loan - Non-Payment Difficulties
j0 = target0["ORGANIZATION_TYPE"].value_counts()
df = pd.DataFrame({'labels': j0.index,'values': j0.values})
df.iplot(kind='pie',labels='labels',values='values', title='Types of Organizations who applied for loan - Non-Payment Difficulties', hole = 0.5)

In [ ]:
# Types of Organizations who applied for loan - Payment Difficulties
j1 = target1["ORGANIZATION_TYPE"].value_counts()
df = pd.DataFrame({'labels': j1.index,'values': j1.values})
df.iplot(kind='pie',labels='labels',values='values', title='Types of Organizations who applied for loan - Payment Difficulties', hole = 0.5)

In [ ]:
# Types of Organizations who applied for loan - Non-Payment Difficulties
temp = target0["ORGANIZATION_TYPE"].value_counts()
temp.iplot(kind='bar', xTitle = 'Organization Name', yTitle = "Count", title = 'Types of Organizations who applied for loan - Non-Payment Difficulties ', color = '#4CB391')

In [ ]:
# Types of Organizations who applied for loan - Payment Difficulties
temp = target1["ORGANIZATION_TYPE"].value_counts()
temp.iplot(kind='bar', xTitle = 'Organization Name', yTitle = "Count", title = 'Types of Organizations who applied for loan - Payment Difficulties ', color = '#4CB391')

We can observe no major changes here.

## 6 Univariate Analysis of Numerical Variables on the basis of 'Target' Variable

In [ ]:
# Function for univariate plots
def uni(df,col,hue =None):
    sns.set(style="darkgrid")
    fig, ax=plt.subplots(nrows =1,ncols=2,figsize=(20,8))
    ax[0].set_title("Distribution Plot")
    sns.distplot(df[~df[col].isna()][col],ax=ax[0], color="#4CB391")
    plt.yscale('log')
    ax[1].set_title("Box Plot")
    sns.boxplot(data =df, x=col,ax=ax[1],orient='v', color="#4CB391")
    plt.yscale('log')
    plt.show()

#### Loan Annuity

In [ ]:
# Distribution and Box plot for 'AMT_ANNUITY' for Loan Non-Payment Difficulties
uni(df=target0,col='AMT_ANNUITY')

In [ ]:
# Distribution and Box plot for 'AMT_ANNUITY' for Loan- Payment Difficulties
uni(df=target1,col='AMT_ANNUITY')

We can observe some outliers and the first quartile is bigger than third quartile for annuity amount which means most of the annuity clients are from first quartile

#### Credit Amount

In [ ]:
# Distribution and Box plot for 'AMT_CREDIT' for Loan Non-Payment Difficulties
uni(df=target0,col='AMT_CREDIT')

In [ ]:
# Distribution and Box plot for 'AMT_CREDIT' for Loan Payment Difficulties
uni(df=target1,col='AMT_CREDIT')

We can observe some outliers and the first quartile is bigger than third quartile for annuity amount which means most of the annuity clients are from first quartile. The distribution curve does not appear to be normal or bell curve.

#### Goods Price

In [ ]:
# Distribution and Box plot for 'AMT_GOODS_PRICE' for Loan- Non-Payment Difficulties
uni(df=target0,col='AMT_GOODS_PRICE')

In [ ]:
# Distribution and Box plot for 'AMT_GOODS_PRICE' for Loan- Non-Payment Difficulties
uni(df=target0,col='AMT_GOODS_PRICE')

We can observe some outliers and the first quartile is bigger than third quartile for annuity amount which means most of the annuity clients are from first quartile

## 6.1 Univariate continuous variable analysis


In [ ]:
# function to dist plot for continuous variables
def plotunidist(var):

    plt.style.use('ggplot')
    sns.despine
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(15,5))
    
    sns.distplot(a=target0[var],ax=ax1)

    ax1.set_title(f'Distribution of {var} for Non-Defaulters',fontsize=15)
            
    sns.distplot(a=target1[var],ax=ax2)
    ax2.set_title(f'Distribution of {var} for Defaulters',fontsize=15)    
        
    plt.show()

In [ ]:
plotunidist('DAYS_EMPLOYED')

In [ ]:
target1['CNT_FAM_MEMBERS'].value_counts()

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
target0['CNT_FAM_MEMBERS'].plot.hist(bins=range(15))
plt.title('Distribution of CNT_FAM_MEMBERS for Non-Defaulters',fontsize=15)
plt.xlabel('CNT_FAM_MEMBERS')
plt.ylabel('LOAN APPLICATION COUNT')

plt.subplot(1, 2, 2)
target1['CNT_FAM_MEMBERS'].plot.hist(bins=range(15))
plt.title(f'Distribution of CNT_FAM_MEMBERS for Defaulters',fontsize=15)
plt.xlabel('CNT_FAM_MEMBERS')
plt.ylabel('LOAN APPLICATION COUNT')  

plt.show()

> We can see that a family of 3 applies loan more often than the other families

# PART D : BIVARIATE ANALYSIS
----------------------------------------------------------
>Bivariate/multivariate analysis involves analysing two or more variables at the same time. 
>
>These analyses yield very specific insights about a data set. You can infer various findings through bivariate analysis.

#### 7.1 Bivariate Analysis of Categorical vs Numerical Variables

#### Analysis - 1

In [ ]:
# 'NAME_EDUCATION_TYPE' vs 'AMT_CREDIT' for Loan - Non Payment Difficulties
fig = px.box(target0, x="NAME_EDUCATION_TYPE", y="AMT_CREDIT", color='NAME_FAMILY_STATUS',
             title="Credit amount vs Education of Loan- Non Payment Difficulties")
fig.show()

In [ ]:
# 'NAME_EDUCATION_TYPE' vs 'AMT_CREDIT' for Loan Payment Difficulties
fig = px.box(target1, x="NAME_EDUCATION_TYPE", y="AMT_CREDIT", color='NAME_FAMILY_STATUS',
                title="Credit amount vs Education of Loan Payment Difficulties")

fig.show()

The graphs for Loan Payment Difficulties and Loan Non-Payment Difficulties apperas to be similar. We observe that Family status of 'civil marriage', 'marriage' and 'separated' of Academic degree education are having higher number of credits than others. Most of the outliers are from Education type 'Higher education' and 'Secondary'. Civil marriage for Academic degree is having most of the credits in the third quartile.

#### Analysis - 2

In [ ]:
# 'AMT_INCOME_RANGE' vs 'AMT_CREDIT' for Loan - Non Payment Difficulties
fig = px.box(target0, x="AMT_INCOME_RANGE", y="AMT_CREDIT", color='NAME_FAMILY_STATUS',
                title="Income range vs Credit amount of Loan Non- Payment Difficulties")

fig.show()

In [ ]:
# 'AMT_INCOME_RANGE' vs 'AMT_CREDIT' for Loan Payment Difficulties
fig = px.box(target1, x="AMT_INCOME_RANGE", y="AMT_CREDIT", color='NAME_FAMILY_STATUS',
                title="Income range vs Credit amount of Loan Payment Difficulties")

fig.show()

The graphs for Loan Payment Difficulties and Loan- Non Payment Difficulties apperas to be similar. We observe that Family status of 'single', 'seperated' and 'married' of income range veryhigh are having higher number of credits than others.

#### 7.2 Bivariate Analysis of Categorical-Categorical

In [ ]:
# Function for bivariate plots
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def biplot(df,feature,title):
    temp = df[feature].value_counts()
    
    # Calculate the percentage of target=1 per category value
    perc = df[[feature, 'TARGET']].groupby([feature],as_index=False).mean()
    perc.sort_values(by='TARGET', ascending=False, inplace=True)

    fig = make_subplots(rows=1, cols=2,subplot_titles=("Count of "+ title,"% of Loan Payment difficulties within each category"))


    fig.add_trace(go.Bar(x=temp.index, y=temp.values),row=1, col=1)
    fig.add_trace(go.Bar(x=perc[feature].to_list(), y=perc['TARGET'].to_list()),row=1, col=2)
    fig['layout']['xaxis']['title']=feature
    fig['layout']['xaxis2']['title']=feature
    fig['layout']['yaxis']['title']='Count'
    fig['layout']['yaxis2']['title']='% of Loan Payment Difficulties'
    fig.update_layout(height=600, width=1000, title_text=title, showlegend=False)
    fig.show()

In [ ]:
# Distribution of income range and the category with maximum Loan-Payment Difficulties
biplot(application_data ,'AMT_INCOME_RANGE','Income range')

From the plot above we can say that clients with 'LOW' Income range have maximum % of Loan-Payment Difficulties.

In [ ]:
# Distribution of income range and the category with maximum Loan-Payment Difficulties
biplot(application_data ,'NAME_INCOME_TYPE','Income type')

From the plot above we can say that clients with 'Maternity leave' Income type have maximum % of Loan-Payment Difficulties.

In [ ]:
# Distribution of Contract Type and the category with maximum Loan-Payment Difficulties
biplot(application_data ,'NAME_CONTRACT_TYPE','Contract type')

From the plot above we can say that clients with 'Cash loans' contract type have maximum % of Loan-Payemnt Difficulties.

In [ ]:
# Distribution of Education Type and the category with maximum Loan-Payment Difficulties
biplot(application_data ,'NAME_EDUCATION_TYPE','Education type')

From the plot above we can say that clients with 'Lower secondary' education type have maximum % of Loan-Payment Difficulties.

In [ ]:
# Distribution of Occupation Type and the category with maximum Loan-Payment Difficulties
biplot(application_data ,'OCCUPATION_TYPE','Occupation type')

From the plot above we can say that clients with 'Lower skill Laborers' occupation type have maximum % of Loan-Payment Difficulties.

In [ ]:
table= pd.pivot_table(application_data, values='TARGET', index=['CODE_GENDER','AMT_INCOME_RANGE'],columns=['NAME_EDUCATION_TYPE'], aggfunc=np.mean)

In [ ]:
table

- From Female category Clients who have LOW income and ACADEMIC DEGREE education have maximum % of Loan-Payment Difficulties
- From Male category Clients wo have MEDIUM income and LOWER SECONDARY education have maximum % of Loan-Payment Difficulties

### 7.3 Bivariate Analysis of Numerical vs Numerical Variables

In [ ]:
#Pairplot for Target 0 (Loan-Non Payment Difficulties)
pair = target0[['AMT_CREDIT', 'AMT_ANNUITY', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE', 'DAYS_BIRTH']].fillna(0)
sns.pairplot(pair)

plt.show()

In [ ]:
#Pairplot for Target 1 (Loan-Payment Difficulties)
pair = target1[['AMT_CREDIT', 'AMT_ANNUITY', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE', 'DAYS_BIRTH']].fillna(0)
sns.pairplot(pair)

plt.show()

In [ ]:
# Heatmap for Loan- Non Payment Difficulties
target0[['AMT_GOODS_PRICE','AMT_INCOME_TOTAL','AMT_ANNUITY','DAYS_EMPLOYED','DAYS_BIRTH',
         'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
         'AMT_CREDIT',]].corr(method = 'pearson').iplot(kind='heatmap',colorscale="blues",title="Correlation Heatmap of Loan- Non Payment Difficulties")

In [ ]:
# Heatmap for Loan-Payment Difficulties
target1[['AMT_GOODS_PRICE','AMT_INCOME_TOTAL','AMT_ANNUITY','DAYS_EMPLOYED',
  'DAYS_BIRTH', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
  'AMT_CREDIT']].corr(method = 'pearson').iplot(kind='heatmap',colorscale="Blues",title="Correlation Heatmap of Loan-Payment Difficulties")


We observe that there is a high correlation between credit amount and goods price. There appears to be some deviancies in the correlation of Loan-Payment Difficulties and Loan- Non Payment Difficulties such as credit amount v/s income.

### 7.4 Bivariate Analysis of numerical variables

In [ ]:
# function for scatter plot for continuous variables
def plotbivar(var1,var2):

    plt.style.use('ggplot')
    sns.despine
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(20,6))
    
    sns.scatterplot(x=var1, y=var2,data=target0,ax=ax1)
    ax1.set_xlabel(var1)    
    ax1.set_ylabel(var2)
    ax1.set_title(f'{var1} vs {var2} for Non-Defaulters',fontsize=15)
    
    sns.scatterplot(x=var1, y=var2,data=target1,ax=ax2)
    ax2.set_xlabel(var1)    
    ax2.set_ylabel(var2)
    ax2.set_title(f'{var1} vs {var2} for Defaulters',fontsize=15)
            
    plt.show()

In [ ]:
plotbivar('AMT_CREDIT','CNT_FAM_MEMBERS')

> We can see that the density in the lower left corner is similar in both the case, so the people are equally likely to default if the family is small and the AMT_CREDIT is low.
> We can observe that larger families and people with larger AMT_CREDIT default less often

In [ ]:
plotbivar('AMT_GOODS_PRICE','AMT_CREDIT')

**Top 10 Correlation for client with payment difficulties**

In [ ]:
corr=target1[['AMT_GOODS_PRICE','AMT_INCOME_TOTAL','AMT_ANNUITY','DAYS_EMPLOYED',
  'DAYS_BIRTH', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
  'AMT_CREDIT']].corr(method = 'pearson')
corr=corr.where(np.triu(np.ones(corr.shape),k=1).astype(np.bool))
corr_df=corr.unstack().reset_index()

In [ ]:
corr_df.columns = ['VAR1','VAR2','CORRELATION']
corr_df.dropna(subset=['CORRELATION'],inplace=True)
corr_df['CORR_ABS']=corr_df['CORRELATION'].abs()

In [ ]:
corr_df.sort_values('CORR_ABS', ascending=False).head(10)

The above dataframe shows top 10 correlation for clients with payment difficulties

## 8. Data Analysis on Previous Application data

#### 8.1 Data loading and cleaning

In [ ]:
# chnaging the negative values in the columns whose name start with DAYS to positive values.
filter_col = [col for col in previous_application if col.startswith('DAYS')]
previous_application[filter_col]= abs(previous_application[filter_col])

In [ ]:
previous_application.head()

In [ ]:
# XNA and XAP are replaced by NaN
previous_application=previous_application.replace('XNA', np.NaN)
previous_application=previous_application.replace('XAP', np.NaN)

In [ ]:
# Value counts of Contract Status in previous application
previous_application['NAME_CONTRACT_STATUS'].value_counts()

#### 8.2 Some Univariate Analysis on previous application data

In [ ]:
# Contract status of previous application
temp = previous_application["NAME_CONTRACT_STATUS"].value_counts()
df = pd.DataFrame({'labels': temp.index,'values': temp.values})
df.iplot(kind='pie',labels='labels',values='values', title='Contract status of previous application',colors=['#75efff','#7e7e7e','#75e757', '#ea7c96',])

We can observe that majority of loans are approved and very less percentage of loans are unused offer

#### Day of the week

In [ ]:
# On which day clients applied for loan in previous data and their counts
temp = previous_application["WEEKDAY_APPR_PROCESS_START"].value_counts()
temp.iplot(kind='bar', xTitle = 'Weekday', yTitle = "Count", title = 'On which day clients applied for loan in previous data and their counts', colors=['#75a275'])

We observe that there are less number of applicants that come in the weekends.

#### Payment Method

In [ ]:
# Payment method that client choose to pay for the previous application
temp = previous_application["NAME_PAYMENT_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,'values': temp.values})
df.iplot(kind='pie',labels='labels',values='values', title='Payment method that client choose to pay for the previous application',colors=['#75e575','#7e7e7e','#ea7c96',])

99% of the clients chose to pay cash through bank.

#### Reasons of previous application rejection

In [ ]:
# Reasons of previous application rejection
temp = previous_application["CODE_REJECT_REASON"].value_counts()
temp.iplot(kind='bar', xTitle = 'Reason', yTitle = "Count", title = 'Reasons of previous application rejection', colors=['#75a275'])

We observe that HC is the reason majority of applications got rejected.

#### Client Type

In [ ]:
# Was the client old or new client when applying for the previous application
temp = previous_application["NAME_CLIENT_TYPE"].value_counts()
df = pd.DataFrame({'labels': temp.index,'values': temp.values})
df.iplot(kind='pie',labels='labels',values='values', title='Was the client old or new client when applying for the previous application',colors=['#ea7c96','#75f445','#75e575',])

We obeserve that majority of the clients are repeaters.

#### Was the previous application for CASH, POS, CAR, CARDS

In [ ]:
# Was the previous application for CASH, POS, CAR, CARDS
temp = previous_application["NAME_PORTFOLIO"].value_counts()
df = pd.DataFrame({'labels': temp.index,'values': temp.values})
df.iplot(kind='pie',labels='labels',values='values', title='Was the previous application for CASH, POS, CAR, CARDS',colors=['#75efff','#7e7e7e','#75e757', '#ea7c96','#fe2345'])

We observe that majority of previous application is for POS and a good amount of it is for cash.

#### Type of goods

In [ ]:
# What kind of goods did the client apply for in the previous application
temp = previous_application["NAME_GOODS_CATEGORY"].value_counts()
temp.iplot(kind='bar', xTitle = 'GOODS CATEGORY', yTitle = "Count", title = 'What kind of goods did the client apply for in the previous application', colors=['#75a275'])

We observe that the majority of loans are for mobiles, consumer electronics, computers and furnitures.

#### Univariate analysis of numerical columns

In [ ]:
uni(df=previous_application,col='AMT_ANNUITY')

We observe that there are some outliers and the curve is not normal or a bell curve

In [ ]:
uni(df=previous_application,col='AMT_CREDIT')

We observe that there are some outliers and the curve is not normal or a bell curve

In [ ]:
uni(df=previous_application,col='AMT_GOODS_PRICE')

We observe that there are some outliers and the curve is not normal or a bell curve

#### 8.3 Bivariate analysis after combining previous and current data

In [ ]:
# Taking SK_ID_CURR and TARGET column from application data for further analysis
appdata_new=application_data[['SK_ID_CURR','TARGET']]

In [ ]:
appdata_new

In [ ]:
# merging application and previous application data on 'SK_ID_CURR'
appdata_merge = appdata_new.merge(previous_application,on='SK_ID_CURR', how='inner')
appdata_merge.shape

In [ ]:
# Distribution of Contract Status and its category with maximum % of Loan-Payment Difficulties
biplot(appdata_merge,'NAME_CONTRACT_STATUS', 'CONTRACT STATUS')

From the first graph it can be seen that most of the contracts from previous application have been Approved

It can be clearly seen from the second graph that the 
- 'Refused' contracts from previous application are the ones who have maximum % of Loan-Payment Difficulties from current application.
- 'Approved' contracts from previous application are the ones who have minimum % of Loan-Payment Difficulties from current application. 

In [ ]:
# Distribution of Contract Status and its category with maximum % of Loan-Payment Difficulties 
biplot(appdata_merge,'NAME_CONTRACT_TYPE', 'CONTRACT TYPE')

From the first graph it can be seen that most of the contract type from previous application was 'Cash loans'

It can be clearly seen from the second graph that the 
- 'Revolving Loans' contracts from previous application are the ones who have maximum % of Loan-Payment Difficulties from current application.
- 'Consumer loans' contracts from previous application are the ones who have minimum % of Loan-Payment Difficulties from current application. 

In [ ]:
# Distribution of Payment Type and its category with maximum % of Loan-Payment Difficulties
biplot(appdata_merge,'NAME_PAYMENT_TYPE', 'PAYMENT TYPE')

From the first graph it can be seen that most of the payment type from previous application was 'Cash through bank'

It can be clearly seen from the second graph that all three types of payments from the previous application have almost same % of Loan-Payment Difficulties from current application 

In [ ]:
# Distribution of Client Type and its category with maximum % of Loan-Payment Difficulties 
biplot(appdata_merge,'NAME_CLIENT_TYPE', 'CLIENT TYPE')

From the first graph it can be seen that most of the clients from previous application are 'Repeater'

It can be clearly seen from the second graph that the 
- 'New' clients from previous application are the ones who have maximum % of Loan-Payment Difficulties from current application.
- 'Refreshed' clients from previous application are the ones who have minimum % of Loan-Payment Difficulties from current application. 

In [ ]:
# Distribution of Cash Loan Purpose and its category with maximum % of Loan-Payment Difficulties 
biplot(appdata_merge,'NAME_CASH_LOAN_PURPOSE', 'CASH LOAN PURPOSE')

From the first graph it can be seen that purpose of cash loan from previous data was maximum for 'Repairs' 

It can be clearly seen from the second graph that the 
- 'Refusal to name the goal' for cash loan from previous application are the ones who have maximum % of Loan-Payment Difficulties from current application.

In [ ]:
table = pd.pivot_table(appdata_merge, values='TARGET', index=['NAME_CLIENT_TYPE'],columns=['NAME_CONTRACT_STATUS'], aggfunc=np.mean)

In [ ]:
table

In [ ]:
ax = table.T.plot(kind='bar')
ylab = ax.set_ylabel('% of Loan-Payment Difficulties')
plt.title('% of Loan Payment Difficulties for NAME_CONTRACT_STATUS and NAME_CLIENT_TYPE', fontdict={'fontsize':20})
plt.show()

It can be observed from the above graph that Client who where 'New' and had 'Cancelled' previous application tend to have more % of Loan-Payment Difficulties in current application 

In [ ]:
table = pd.pivot_table(appdata_merge, values='TARGET', index=['NAME_CONTRACT_TYPE'],columns=['NAME_CONTRACT_STATUS'], aggfunc=np.mean)

In [ ]:
table

In [ ]:
ax = table.T.plot(kind='bar')
ylab = ax.set_ylabel('% of Loan-Payment Difficulties')
plt.title('% of Loan Payment Difficulties for NAME_CONTRACT_STATUS and NAME_CONTRACT_TYPE', fontdict={'fontsize':20})
plt.show()

It can be observed from the above graph that Clients with 'Revolving loans' and with 'Refused' previous application tend to have more % of Loan-Payment Difficulties in current application 

# PART E : SUMMARY OF THE CASE STUDY
----------------------------------------------------------

## 9. Insights

### 9.1 Application Data

- The count of ‘Maternity Leave’ in ‘NAME_INCOME_TYPE’ is very less and it also has maximum % of payment difficulties- around 40%. Hence, client with income type as ‘Maternity leave’ are the driving factors for Loan Defaulters.

- The count of ‘Low skilled Laborers’ in ‘OCCUPATION_TYPE’ is comparatively very less and it also has maximum % of payment difficulties- around 17%. Hence, client with occupation type as ‘Low skilled Laborers’ are the driving factors for Loan Defaulters.

- The count of ‘Lower Secondary’ in ‘NAME_EDUCATION_TYPE’ is comparatively very less and it also has maximum % of payment difficulties- around 11%. Hence, client with education type as ‘Lower Secondary’ are the driving factors for Loan Defaulters.


### 9.2 Previous Application Data

- The count of ‘Refusal to name the goal’ in ‘NAME_CASH_LOAN_PURPOSE’ is comparatively very less and it also has maximum % of payment difficulties- around 23%. Hence, clients who have ‘Refused to name the goal’ for cash loan in previous application are the driving factors for Loan Defaulters.

- The count of ‘Refused’ in ‘NAME_CONTRACT_STATUS’ is comparatively less and it also has maximum % of payment difficulties- around 12%. Hence, client with contract status as ‘Refused’ in previous application are the driving factors for Loan Defaulters.

- The count of ‘Revolving Loans’ in ‘NAME_CONTRACT_TYPE’ is comparatively very less and it also has maximum % of payment difficulties- around 10%. Hence, client with contract type as ‘Revolving loans’ in previous application are the driving factors for Loan Defaulters.

- It can be observed from the graph that Clients with 'Revolving loans' and with 'Refused' previous application tend to have more % of payment difficulties in current application. Since the count of both 'Revolving loans' and ‘Refused’ is comparatively less(from the graphs in previous slide), clients with ‘Revolving Loans’ and ‘Refused’ previous application are driving factors for Loan Defaulters